In [ ]:
import numpy as np
import pandas as pd
import os
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
path = os.path.join("../../data", "integrated_data_20251030_185440.csv")

In [ ]:
data = pd.read_csv('integrated_data_20251030_185440.csv')
#data['Date'] = pd.to_datetime(data['Date'])
#data.set_index('Date', inplace=True)

env_cols = ['env0', 'env1', 'env2', 'env3']
finger_cols = ['thumb_tip','thumb_base','index','middle','ring','pinky']

data[env_cols] = data[env_cols].astype(float)
data[finger_cols] = data[finger_cols].astype(float)

scaler = MinMaxScaler(feature_range=(0, 1)) #Scales data to be between 0 and 1
env_scaled = scaler.fit_transform(data[env_cols])
finger_scaled = scaler.fit_transform(data[finger_cols])
